In [1]:
from collections import OrderedDict
import torch

import numpy as np
from torchvision.models import resnet18

In [2]:
ckpt=torch.load('resnet18-f37072fd.pth')

In [3]:
m=resnet18(pretrained=True)
m.load_state_dict(ckpt)
_=m.eval()
torch.set_grad_enabled(False)

/home/lyan/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/lyan/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [21]:
def write_graph(model, output_name):
#     submodules=list(model.modules())
    
    f = open(output_name, 'wb')
    n=3
    
    header = np.zeros(256, dtype=np.int32)
    header[0]=42
    header[1]=1
    header[2] = n

    f.write(header.tobytes())
    offset=256
    for i, l in enumerate(model.modules()):
        print(i,type(l), offset)
            
        if type(l) == torch.nn.Conv2d:
            header = np.zeros(256, dtype=np.int32)
            header[0]=1 # type for conv
            header[1]=l.in_channels; header[2]=l.out_channels
            header[3]=l.kernel_size[0]; header[4]=l.kernel_size[1]
            header[5]=l.stride[0];header[6]=l.stride[1]
            
            header[7]=l.padding[0];header[8]=l.padding[1]
            if l.bias is None:
                header[9]=0
            else:
                header[9]=int(l.bias)
            header[10]=l.groups
            header[11]=l.dilation[0];header[12]=l.dilation[1];
            
            weight=l.weight.detach().to(torch.float32).numpy()
            for i in range(len(weight.shape)):
                header[13+i] = weight.shape[i]

            offset += len(header)
            offset += len(weight.reshape(-1))
            print(weight.reshape(-1)[0:10])
            f.write(header.tobytes())
            f.write(weight.tobytes())
            if l.bias is not None or l.bias:
                f.write(l.bias.detach().to(torch.float32).numpy().tobytes())
            
        elif type(l) == torch.nn.BatchNorm2d:
            header = np.zeros(256, dtype=np.int32)
            header[0]=2 # type for conv
            header[1] = l.num_features
            
            f.write(header.tobytes())
            f.write(l.weight.detach().to(torch.float32).numpy().tobytes())
            f.write(l.bias.detach().to(torch.float32).numpy().tobytes())
            f.write(l.running_mean.detach().to(torch.float32).numpy().tobytes())
            f.write(l.running_var.detach().to(torch.float32).numpy().tobytes())
            offset += 256
            offset += l.weight.shape[0]*4
        elif type(l) == torch.nn.ReLU:
            header = np.zeros(256, dtype=np.int32)
            header[0]=3 # type for conv
            f.write(header.tobytes())
            offset+=256
        elif type(l) == torch.nn.MaxPool2d:
            header = np.zeros(256, dtype=np.int32)
            header[0]=4
            header[1]=l.kernel_size
            header[2]=l.stride
            header[3]=l.padding
            header[4]=l.dilation
            header[5] = 102
            f.write(header.tobytes())
            offset+=256
        elif type(l) == torch.nn.AdaptiveAvgPool2d:
            header = np.zeros(256, dtype=np.int32)
            header[0]=5
            header[1]=l.output_size[0]
            header[2]=l.output_size[1]
            f.write(header.tobytes())
        elif type(l) == torch.nn.Linear:
            header = np.zeros(256, dtype=np.int32)
            header[0]=6
            header[1]=l.in_features
            header[2]=l.out_features
            f.write(header.tobytes())
            f.write(l.weight.T.detach().to(torch.float32).numpy().tobytes())
            f.write(l.bias.detach().to(torch.float32).numpy().tobytes())
        else:
#             print(type(l))
            continue
    f.close()

In [22]:
write_graph(m, 'r18.bin')

0 <class 'torchvision.models.resnet.ResNet'> 256
1 <class 'torch.nn.modules.conv.Conv2d'> 256
[-0.01041935 -0.00613561 -0.00180978  0.07484142  0.05661485  0.01708333
 -0.01269388  0.01108271  0.00952757 -0.10992692]
2 <class 'torch.nn.modules.batchnorm.BatchNorm2d'> 9920
3 <class 'torch.nn.modules.activation.ReLU'> 10432
4 <class 'torch.nn.modules.pooling.MaxPool2d'> 10688
5 <class 'torch.nn.modules.container.Sequential'> 10944
6 <class 'torchvision.models.resnet.BasicBlock'> 10944
7 <class 'torch.nn.modules.conv.Conv2d'> 10944
[ 0.05759342 -0.09511436 -0.02027232 -0.07455588 -0.799308   -0.21283598
  0.06557069 -0.09653367 -0.01211061 -0.00699444]
8 <class 'torch.nn.modules.batchnorm.BatchNorm2d'> 48064
9 <class 'torch.nn.modules.activation.ReLU'> 48576
10 <class 'torch.nn.modules.conv.Conv2d'> 48832
[ 0.02594677 -0.10457563 -0.00477124 -0.08622317 -0.33020768 -0.10275265
 -0.05742571 -0.19074456 -0.05464623 -0.01695126]
11 <class 'torch.nn.modules.batchnorm.BatchNorm2d'> 85952
12 <c

In [201]:
def write_data(x, name, output_name):
    with open(output_name, 'wb') as f:
        header = np.zeros(256, dtype=np.int32)
        header[0]=42
        header[1]=1
        for i in range(len(x.shape)):
            header[2+i] = x.shape[i]
        
        f.write(header.tobytes())
        
        f.write(x.tobytes())

In [6]:
import cv2

In [7]:
img=cv2.imread('/home/lyan/Downloads/000000062491.jpg')
# img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img=cv2.resize(img, (64,64))
img=torch.tensor(img).reshape(1,3,64,64).to(torch.float32) / 255

In [189]:
img.reshape(-1)[0:10]

tensor([0.4039, 0.5255, 0.5216, 0.4549, 0.5412, 0.5451, 0.4667, 0.5725, 0.5529,
        0.4784])

In [11]:
x=m.conv1(img)
x=m.bn1(x)
x=m.relu(x)

In [12]:
x2=m.maxpool(x)

In [13]:
x3=m.layer1(x2)

In [14]:
x4=m.layer2(x3)

In [15]:
x5=m.layer3(x4)

In [16]:
x6=m.layer4(x5)

In [17]:
x7=m.avgpool(x6)

In [8]:
out=m(img)

In [10]:
out.reshape(-1)[0:10]

tensor([ 1.8758,  1.3773, -1.9276, -1.4540, -1.7166,  0.2312, -1.0988, -2.5033,
        -2.5376, -1.9886])

In [ ]:
M=1
N=512
K=1000

In [20]:
m.fc.weight.shape

torch.Size([1000, 512])

In [19]:
x7.shape

torch.Size([1, 512, 1, 1])

In [18]:
(torch.matmul(x7.squeeze(), m.fc.weight.T)).reshape(-1)[0:10]

tensor([ 1.8784,  1.3743, -1.9283, -1.4271, -1.7230,  0.2179, -1.0876, -2.5240,
        -2.5339, -1.9762])

In [268]:
(torch.matmul(x7.squeeze(), m.fc.weight.T)+m.fc.bias).reshape(-1)[0:10]

tensor([ 1.8758,  1.3773, -1.9276, -1.4540, -1.7166,  0.2312, -1.0988, -2.5033,
        -2.5376, -1.9886])

In [35]:
import matplotlib.pyplot as plt

In [203]:
write_data(x2.numpy(), '', 'x2.bin')

In [170]:
write_data(img.numpy(), '', 'test_img.bin')
write_data(ckpt['conv1.weight'].detach().to(torch.float32).numpy(), 'conv1.weight', 'test.bin')

In [202]:
y=torch.nn.functional.conv2d(img, ckpt['conv1.weight'].detach() , )